In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-set/india_df.csv
/kaggle/input/data-set/italy_df.csv
/kaggle/input/data-set/my_folder/dataset anemia/Dataset anemia.docx
/kaggle/input/data-set/my_folder/dataset anemia/Italy/Italy.xlsx
/kaggle/input/data-set/my_folder/dataset anemia/Italy/7/007_palpebral.png
/kaggle/input/data-set/my_folder/dataset anemia/Italy/7/7.jpg
/kaggle/input/data-set/my_folder/dataset anemia/Italy/7/007_forniceal.png
/kaggle/input/data-set/my_folder/dataset anemia/Italy/7/007_forniceal_palpebral.png
/kaggle/input/data-set/my_folder/dataset anemia/Italy/47/T_16_20190608_075548_forniceal.png
/kaggle/input/data-set/my_folder/dataset anemia/Italy/47/T_16_20190608_075548_palpebral.png
/kaggle/input/data-set/my_folder/dataset anemia/Italy/47/T_16_20190608_075548.jpg
/kaggle/input/data-set/my_folder/dataset anemia/Italy/47/T_16_20190608_075548_forniceal_palpebral.png
/kaggle/input/data-set/my_folder/dataset anemia/Italy/17/017_forniceal_palpebral.png
/kaggle/input/data-set/my_folder/dataset anemia/

In [2]:
import pandas as pd

In [3]:
india_df=pd.read_csv("/kaggle/input/data-set/india_df.csv")
italy_df=pd.read_csv("/kaggle/input/data-set/italy_df.csv")

In [4]:
india_folder_path="/kaggle/input/data-set/my_folder/dataset anemia/India"
italy_folder_path="/kaggle/input/data-set/my_folder/dataset anemia/Italy"

In [5]:
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)


In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam,AdamW
from sklearn.model_selection import train_test_split
import pandas as pd
import cv2

# Function to load images and labels from the directory structure
def load_images_and_labels(base_dir, df):
    images = []
    labels = []

    # Iterate over each row in the DataFrame to get the person information
    for _, row in df.iterrows():
        person_number = row['Number']  # Assuming 'Number' is the identifier
        label = row['Class Label']     # Assuming 'Class Label' indicates anemia (1) or non-anemia (0)

        # Path to the folder for this person
        person_folder_path = os.path.join(base_dir, str(person_number))

        # Check if the folder exists
        if os.path.isdir(person_folder_path):
            # Load all images of this person from the folder
            for file in os.listdir(person_folder_path):
                if file.endswith(('.png')):  # Adjust based on your image formats
                    img_path = os.path.join(person_folder_path, file)

                    # Load and preprocess the image
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, (224, 224))  # Resize to VGG16 input size
                    img = np.array(img, dtype='float32')

                    # Append the image and the label
                    images.append(img)
                    labels.append(label)

    return np.array(images), np.array(labels)


# Load India dataset
X_india, y_india = load_images_and_labels(india_folder_path, india_df)

# Load Italy dataset
X_italy, y_italy = load_images_and_labels(italy_folder_path, italy_df)

# Combine India and Italy datasets
X_combined = np.concatenate([X_india, X_italy], axis=0)
y_combined = np.concatenate([y_india, y_italy], axis=0)

# Normalize the combined images
X_combined = preprocess_input(X_combined)

# Split the combined dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.15, random_state=42)

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.metrics import Precision, Recall, Accuracy
from tensorflow.keras.regularizers import l2

# Load VGG16 base model with pre-trained weights
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers of the base model except the last three
for layer in base_model.layers:  # Freeze all layers except the last three
    layer.trainable = False
# Set the last VGG block to trainable
base_model.layers[-2].trainable = True
base_model.layers[-3].trainable = True
base_model.layers[-4].trainable = True

# Initialize the model
model = Sequential()

# Add the VGG16 base model
model.add(base_model)

# Add the Global Average Pooling layer
model.add(Flatten())
# Add a Global Average Pooling layer
#model.add(GlobalAveragePooling2D())

# Deeper classification head
model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
##model.add(BatchNormalization())
model.add(Dropout(0.3))  # Stronger regularization

model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.008)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))  # Final binary classification layer

# Compile the model with accuracy, precision, and recall metrics
model.compile(
    optimizer=AdamW(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(), Recall()]  # Add precision and recall
)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), shuffle=False)

# Evaluate the model
test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test)

# Print evaluation metrics
print(f"Test Accuracy: {test_acc:.2f}")
print(f"Test Precision: {test_precision:.2f}")
print(f"Test Recall: {test_recall:.2f}")

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 435ms/step - accuracy: 0.5998 - loss: 19.3908 - precision_13: 0.4691 - recall_13: 0.6120 - val_accuracy: 0.6907 - val_loss: 17.8699 - val_precision_13: 0.5686 - val_recall_13: 0.7838
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.7517 - loss: 17.0327 - precision_13: 0.6366 - recall_13: 0.7753 - val_accuracy: 0.7526 - val_loss: 15.9373 - val_precision_13: 0.6327 - val_recall_13: 0.8378
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.7777 - loss: 15.4889 - precision_13: 0.6592 - recall_13: 0.8309 - val_accuracy: 0.7423 - val_loss: 14.6479 - val_precision_13: 0.6765 - val_recall_13: 0.6216
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.7887 - loss: 14.3025 - precision_13: 0.6865 - recall_13: 0.7958 - val_accuracy: 0.8041 - val_loss: 13.5939 - val_precision_13: 0.8214 - val_recall_13: 0.6216
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.8407 - loss: 13.3382 - precision_13:

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.metrics import Precision, Recall, Accuracy
from tensorflow.keras.regularizers import l2

# Load VGG16 base model with pre-trained weights
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers of the base model except the last three
for layer in base_model.layers:  # Freeze all layers except the last three
    layer.trainable = False
# Set the last VGG block to trainable
base_model.layers[-2].trainable = True
base_model.layers[-3].trainable = True
base_model.layers[-4].trainable = True

# Initialize the model
model = Sequential()

# Add the VGG16 base model
model.add(base_model)

# Add the Global Average Pooling layer
model.add(Flatten())
# Add a Global Average Pooling layer
#model.add(GlobalAveragePooling2D())

# Deeper classification head
model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.008)))
##model.add(BatchNormalization())
model.add(Dropout(0.3))  # Stronger regularization

model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.008)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.008)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))  # Final binary classification layer

# Compile the model with accuracy, precision, and recall metrics
model.compile(
    optimizer=AdamW(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(), Recall()]  # Add precision and recall
)

# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), shuffle=False)

# Evaluate the model
test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test)

# Print evaluation metrics
print(f"Test Accuracy: {test_acc:.2f}")
print(f"Test Precision: {test_precision:.2f}")
print(f"Test Recall: {test_recall:.2f}")

Epoch 1/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 429ms/step - accuracy: 0.6465 - loss: 16.4000 - precision_10: 0.5179 - recall_10: 0.6721 - val_accuracy: 0.7526 - val_loss: 14.9051 - val_precision_10: 0.7407 - val_recall_10: 0.5405
Epoch 2/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.7855 - loss: 14.4407 - precision_10: 0.6667 - recall_10: 0.8445 - val_accuracy: 0.7526 - val_loss: 13.4897 - val_precision_10: 0.7826 - val_recall_10: 0.4865
Epoch 3/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.7815 - loss: 13.1551 - precision_10: 0.6613 - recall_10: 0.8374 - val_accuracy: 0.7835 - val_loss: 12.4848 - val_precision_10: 0.8333 - val_recall_10: 0.5405
Epoch 4/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.8038 - loss: 12.2160 - precision_10: 0.6940 - recall_10: 0.8435 - val_accuracy: 0.7732 - val_loss: 11.7052 - val_precision_10: 0.8000 - val_recall_10: 0.5405
Epoch 5/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.8098 - loss: 11.5272 - precision_10: 0.69

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.metrics import Precision, Recall, Accuracy
from tensorflow.keras.regularizers import l2

# Load VGG16 base model with pre-trained weights
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers of the base model except the last three
for layer in base_model.layers:  # Freeze all layers except the last three
    layer.trainable = False
# Set the last VGG block to trainable
base_model.layers[-2].trainable = True
base_model.layers[-3].trainable = True
base_model.layers[-4].trainable = True

# Initialize the model
model = Sequential()

# Add the VGG16 base model
model.add(base_model)

# Add the Global Average Pooling layer
model.add(Flatten())
# Add a Global Average Pooling layer
#model.add(GlobalAveragePooling2D())

# Deeper classification head
model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.008)))
##model.add(BatchNormalization())
model.add(Dropout(0.3))  # Stronger regularization

model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.008)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.008)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))  # Final binary classification layer

# Compile the model with accuracy, precision, and recall metrics
model.compile(
    optimizer=AdamW(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(), Recall()]  # Add precision and recall
)

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), shuffle=False)

# Evaluate the model
test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test)

# Print evaluation metrics
print(f"Test Accuracy: {test_acc:.2f}")
print(f"Test Precision: {test_precision:.2f}")
print(f"Test Recall: {test_recall:.2f}")

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 441ms/step - accuracy: 0.5695 - loss: 16.5373 - precision_14: 0.4385 - recall_14: 0.5815 - val_accuracy: 0.7113 - val_loss: 15.2393 - val_precision_14: 0.5849 - val_recall_14: 0.8378
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.7341 - loss: 14.6012 - precision_14: 0.6100 - recall_14: 0.7826 - val_accuracy: 0.7732 - val_loss: 13.5834 - val_precision_14: 0.7586 - val_recall_14: 0.5946
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.7370 - loss: 13.4134 - precision_14: 0.6082 - recall_14: 0.8173 - val_accuracy: 0.7320 - val_loss: 12.7151 - val_precision_14: 0.9231 - val_recall_14: 0.3243
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - accuracy: 0.7368 - loss: 12.4915 - precision_14: 0.6250 - recall_14: 0.7276 - val_accuracy: 0.7526 - val_loss: 11.9112 - val_precision_14: 0.7241 - val_recall_14: 0.5676
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.8096 - loss: 11.7736 - precision_14: 0.69